# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pymysql

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

# Import data set from Ironhack's database

# # Connect to Ironhack DB
driver   = 'mysql+pymysql:'
user     = 'data-students' 
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136' 
database = 'orders' 


connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

# Check database content
pd.read_sql('SHOW TABLES;', engine)


C:\Users\miner\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 1")
  result = self._query(query)


,Tables_in_orders
0,orders


In [3]:
## Import data
orders_original = pd.read_sql('SELECT * FROM orders;', engine)
orders          = pd.read_sql('SELECT * FROM orders;', engine)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [42]:
orders = orders_original

In [43]:
# your code here

## Aggregate the amount spent per customer
amount = orders.groupby('CustomerID').sum().reset_index()
amount = amount[['CustomerID', 'amount_spent']]
amount

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [44]:
##Find out the quantile ranges requested and label them
labels = ['Regular', 'Preferred', 'VIP']
amount['customer_type'] = pd.qcut(amount['amount_spent'],[0, .75, .95, 1],labels=labels)
amount

,CustomerID,amount_spent,customer_type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Regular
...,...,...,...
4334,18280,180.60,Regular
4335,18281,80.82,Regular
4336,18282,178.05,Regular
4337,18283,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [45]:
## Extract the list of clients. Make sure the client ID are unique

orders2 = orders_original
client_list = orders2[['CustomerID', 'Country']]
client_list.drop_duplicates(subset ='CustomerID', inplace = True) 
client_list


C:\Users\miner\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,CustomerID,Country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,France
46,13748,United Kingdom
65,15100,United Kingdom
...,...,...
394965,13436,United Kingdom
395041,15520,United Kingdom
395777,13298,United Kingdom
396506,14569,United Kingdom


In [46]:
## Add type information to the list
type_client_list = pd.merge(client_list, amount, on = 'CustomerID')
type_client_list

,CustomerID,Country,amount_spent,customer_type
0,17850,United Kingdom,5391.21,Preferred
1,13047,United Kingdom,3237.54,Preferred
2,12583,France,7281.38,VIP
3,13748,United Kingdom,948.25,Regular
4,15100,United Kingdom,876.00,Regular
...,...,...,...,...
4334,13436,United Kingdom,196.89,Regular
4335,15520,United Kingdom,343.50,Regular
4336,13298,United Kingdom,360.00,Regular
4337,14569,United Kingdom,227.39,Regular


In [47]:
## Filter VIP clients
VIP_clients = type_client_list[type_client_list['customer_type'] == 'VIP']
VIP_clients

,CustomerID,Country,amount_spent,customer_type
2,12583,France,7281.38,VIP
8,15311,United Kingdom,60767.90,VIP
12,16029,United Kingdom,81024.84,VIP
14,12431,Australia,6487.45,VIP
15,17511,United Kingdom,91062.38,VIP
...,...,...,...,...
3701,17509,United Kingdom,6115.14,VIP
3752,12536,France,12601.83,VIP
4045,12357,Switzerland,6207.67,VIP
4213,18139,United Kingdom,8438.34,VIP


In [49]:
## Group by country and count the customers per country
VIP_clients_per_country = VIP_clients.groupby('Country',as_index=False).count()
VIP_clients_per_country = VIP_clients_per_country.rename(columns={'CustomerID':'number_clients'})
VIP_clients_per_country.sort_values(by = 'number_clients', ascending = False, inplace = True)
VIP_clients_per_country[['Country', 'number_clients']].head(1)

,Country,number_clients
16,United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [50]:
# your code here
## Filter VIP+preferred clients
VIP_Preferred_clients = type_client_list[(type_client_list['customer_type'] == 'VIP') | (type_client_list['customer_type'] == 'Preferred')]
VIP_Preferred_clients

,CustomerID,Country,amount_spent,customer_type
0,17850,United Kingdom,5391.21,Preferred
1,13047,United Kingdom,3237.54,Preferred
2,12583,France,7281.38,VIP
5,15291,United Kingdom,4668.30,Preferred
6,14688,United Kingdom,5630.87,Preferred
...,...,...,...,...
4148,12950,United Kingdom,1843.00,Preferred
4203,12349,Italy,1757.55,Preferred
4213,18139,United Kingdom,8438.34,VIP
4329,16000,United Kingdom,12393.70,VIP


In [54]:
## Group by country and count the customers per country
VIP_Preferred_clients_per_country = VIP_Preferred_clients.groupby('Country',as_index=False).count()
VIP_Preferred_clients_per_country = VIP_Preferred_clients_per_country.rename(columns={'CustomerID':'number_clients'})
VIP_Preferred_clients_per_country.sort_values(by = 'number_clients', ascending = False, inplace = True)
VIP_Preferred_clients_per_country[['Country', 'number_clients']].head(1)

,Country,number_clients
26,United Kingdom,932
